In [ ]:
import json

data = []
with open("/Users/philip/Documents/PhD/data/ArenaData/arena_fits/2015-07-05.json") as f:
    data = f.readlines()

json_lines = []

for line in data:
    jsline = json.loads(line)
    json_lines.append(jsline)

In [ ]:
import pandas as pd

frame = pd.DataFrame.from_dict(json_lines)

In [ ]:
# rebuild dataframe
# make dataframe of dicts nested in 'value' column
value = pd.DataFrame(list(frame['value']))
del frame['value']

# make dataframe of dicts nested in 'trackeeHistory' column
trackee = pd.DataFrame(list(value['trackeeHistory']))
del value['trackeeHistory']

chi2PerDof = pd.DataFrame(list(trackee['chi2PerDof']))
chi2PerDof.columns = ['chi2PerDof']
probChi2 = pd.DataFrame(list(trackee['probChi2']))
probChi2.columns = ['probChi2']
nMeasurements = pd.DataFrame(list(trackee['nMeasurements']))
nMeasurements.columns = ['nMeasurements']

In [ ]:
# make dataframe with a 'coordinates' column
averagecoordinate = pd.DataFrame(list(value['averagecoordinate']))
coordinates = pd.DataFrame(list(averagecoordinate['avg']))
averagecoordinate = averagecoordinate.join(coordinates)
error = pd.DataFrame(list(averagecoordinate['error']))
errorcoordinates = pd.DataFrame(list(error['coordinates']))
del errorcoordinates[2]
errorcoordinates.columns = ['x_error','y_error']

del averagecoordinate['avg']
del value['averagecoordinate']

# join dataframes
frame = frame.join(value.join(averagecoordinate))
frame = frame.join(chi2PerDof)
frame = frame.join(probChi2)
frame = frame.join(errorcoordinates)
frame = frame.join(nMeasurements)
del frame['regionsNodesIds']
del frame['error']
del frame['type']

In [ ]:
frame = frame[(frame['probChi2'] > 0.3) & 
              (frame['x_error'] < 10) & 
              (frame['y_error'] < 10) & 
              (frame['chi2PerDof'] < 1.2)]

In [ ]:
frame = frame.sort_values(by='measurementTimestamp')

In [ ]:
# get most frequent mac address
macs = frame['sourceMac'].value_counts()
macs[:5]

In [ ]:
from math import floor, ceil, sqrt, pi, exp
import numpy as np
import time

begin = time.time()

# size of binned region (number of bins 3x3m)
width = 80; height = 60
# numbers of time intervals and subwindows
periods = 10; nSubwindows = 4

# total density histogram per period
bins = np.zeros((periods, height, width))

# size of time interval (milliseconds)
interval = 120000/nSubwindows

# sliding window step size
shift = 30000

weights = np.array([0.4, 0.8, 1.2, 1.6]) 
weights = weights[:nSubwindows]

def kernel(u): 
    return 1/sqrt(2*pi) * exp(-(u**2)/2)

for k in range(periods):
    
    # create dictionary with a number of density histograms for each mac address
    # the density histograms are added and normalized in a weighted sum
    # then added to the total density histogram per period
    
    subBins = np.zeros((len(set(frame['sourceMac'])), nSubwindows, height,width))
    
    normalization = np.zeros((len(set(frame['sourceMac'])), nSubwindows))
    
    # dictionary of density histograms using mac addresses as keys
    states = dict(zip(set(frame['sourceMac']), zip(normalization, subBins)))

    for m in range(nSubwindows):
        
        #print('Subwindow:', m)

        start = min(frame['measurementTimestamp']) + k * shift + m * interval
        stop = start + interval

        subwindow = frame[(frame['measurementTimestamp'] >= start) & 
                           (frame['measurementTimestamp'] < stop)]
        
        # loop through the measurements (positions) 
        # update appropriate density histogram (mac, subwindow)
        # count number of measurements in normalization factor

        for j in range(len(subwindow)):
            
            # bin positions
            xbin = width/2  + floor(subwindow['coordinates'].values[j][0] / 3)
            ybin = height/2 + floor(subwindow['coordinates'].values[j][1] / 3)

            if xbin >= 0 and xbin < width and ybin >= 0 and ybin < height:

                # count positions for normalization
                states[subwindow['sourceMac'].values[j]][0][m] += 1 

                #### kernel density estimation #########################################

                hx = subwindow['x_error'].values[j]
                hy = subwindow['y_error'].values[j]

                smooth_bins = np.zeros((60,80))

                for u in range(width):
                    for v in range(height):
                        smooth_bins[v][u] += kernel((u - xbin) / hx) * kernel((v - ybin) / hy)

                smooth_bins /= hx * hy

                ######################################################################

                # update density histogram
                states[subwindow['sourceMac'].values[j]][1][m] += smooth_bins

    #### apply weighted sum over subwindows for each mac
    
    for mac in set(frame['sourceMac']):
        if states[mac][0].sum() > 0:
            for w in range(nSubwindows):
                bins[k] += (states[mac][1][w] * weights[w]) / np.multiply(weights, states[mac][0]).sum()
    
    #### write density histogram to file ####

    np.savetxt('output/bins_%d.txt' %  k, bins[k], delimiter=',')

    print('Time window:', k)
    
end = time.time()
print('Time elpased:', end - begin) 

In [ ]:
# this is a plot test cell
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

fig = plt.figure(figsize=(12,8))

width = 80; height = 60

X = np.arange(width)
Y = np.arange(height)*(-1)
Xs, Ys = np.meshgrid(X, Y)

Z = np.loadtxt('output/bins_%d.txt' % 0, delimiter=',')

ax = Axes3D(fig)
ax.plot_surface(Xs, Ys, Z, rstride=2, cstride=1, cmap='hsv')#, cmap='hot')
#ax.set_zlim3d(0, 0.05)
plt.show()

In [ ]:
bins[0].sum()

In [ ]:
bins.shape

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

fig = plt.figure(figsize=(12,8))

X = np.arange(width)
Y = np.arange(height)*(-1)
Xs, Ys = np.meshgrid(X, Y)

for i in range(periods):
    Z = np.loadtxt('output/bins_%d.txt' % i, delimiter=',')

    ax = Axes3D(fig)
    ax.plot_surface(Xs, Ys, Z, rstride=2, cstride=1, cmap='hsv')#, cmap='hot')
    ax.set_zlim3d(0, 0.0005)
    if i < 10:
        number = '000' + str(i)
    elif i > 9:
        number = '00' + str(i)
    elif i > 99:
        number = '0' + str(i)
    plt.savefig('output/surface-%s.png' % number)
#plt.show()


In [ ]:
import matplotlib.pyplot as plt

#im = plt.imread('/Users/philip/Documents/PhD/data-analysis/escience/arena_sensation.png')

#fig = plt.figure()
#ax = fig.add_subplot(1,1,1)
# convenience method:
fig, ax = plt.subplots(figsize=(12,8))

#ax.set_xlim((0,1853)) # 1853
#ax.set_ylim((0,1369)) # 1369

#ax.imshow(im)

for i in range(periods):
    ax.imshow(bins[i], aspect='auto', cmap='hsv', alpha=1)
    if i < 10:
        number = '000' + str(i)
    elif i > 9:
        number = '00' + str(i)
    elif i > 99:
        number = '0' + str(i)
    plt.savefig('output/map-%s.png' % number)
    
#plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, projection='3d')

x_data, y_data = np.meshgrid( np.arange(width),
                              np.arange(height)*(-1) )

col = ['r', 'y', 'c', 'k', 'c','r'] * height * width
# colors = np.random.choice(col, height*width)

x_data = x_data.flatten()
y_data = y_data.flatten()

for i in range(periods):
    z_data = bins[i].flatten()
    ax.set_zlim3d(0, 16)
    ax.bar3d( x_data,
              y_data,
              np.zeros(len(z_data)),
              1, 1, z_data, color=col) # 
    if i < 10:
        number = '000' + str(i)
    elif i > 9:
        number = '00' + str(i)
    elif i > 99:
        number = '0' + str(i)
    plt.savefig('output/bar-%s.png' % number)

#plt.show()